In [1]:
import os

In [2]:
%pwd

'c:\\Users\\KEERTHI RAJ\\Desktop\\KEERTHI RAJ\\PROJECTS\\DLPROJECTS\\Chicken-Disease-Classification-MLOPS\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\KEERTHI RAJ\\Desktop\\KEERTHI RAJ\\PROJECTS\\DLPROJECTS\\Chicken-Disease-Classification-MLOPS'

### Creating the Entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_directory: Path
    source_URL: str
    local_data_file: Path
    unzip_directory: Path

### Creating the Configurations

In [6]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import readYaml, createDirectories

In [7]:
class ConfiguationManager:

    """
    All these values are taken from the config file
    """
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
            ):
        
        self.config = readYaml(config_filepath)
        self.params = readYaml(params_filepath)

        createDirectories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
         config = self.config.data_ingestion
         createDirectories([config.root_directory])

         data_ingestion_config = DataIngestionConfig(
             root_directory = config.root_directory,
             source_URL = config.source_URL,
             local_data_file = config.local_data_file,
             unzip_directory = config.unzip_directory
            )
         
         return data_ingestion_config

### Creating the Components

In [8]:
import urllib.request as request
import zipfile
from ChickenDiseaseClassifier import logger
from ChickenDiseaseClassifier.utils.common import getSize

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers  = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {getSize(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_directory
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

### Creating the Pipeline

In [10]:
try:
    config = ConfiguationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-01 20:38:28,528: INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-06-01 20:38:28,539: INFO:common:yaml file: params.yaml loaded successfully]
[2024-06-01 20:38:28,539: INFO:common:yaml file: params.yaml loaded successfully]
[2024-06-01 20:38:28,541: INFO:common:Created Directory at artifacts]
[2024-06-01 20:38:28,543: INFO:common:Created Directory at artifacts/data_ingestion]
[2024-06-01 20:38:28,545: INFO:583930610:File already exists of size: ~11345 KB]
